# 项目结构
- The_battle_memory
    - Genichiro_Ashina (内含1场演示用数据集和101场训练用的完整数据集，由于文件过大所以只上传演示用数据集)
        - training_data-1.npy (演示用数据集)
        - training_data-2.npy
        - ... (2 ~ 102 都是训练用的完整数据集)
        - training_data-102.npy

- pysekiro
    - \__init__.py

    - keys.py (键盘按键的信息)
    - get_keys.py (捕获键盘的按键)
    - direct_keys.py (控制键盘的按键)
    - actions.py (动作控制)

    - grab_screen.py (屏幕图像抓取)
    - get_vertices.py (顶点位置获取)
    - get_status.py (状态获取)

    - collect_data.py (收集数据)

    - adjustment.py (游戏窗口校准)

    - model.py （模型定义）
    - train_with_dl.py （用深度学习训练）
    - Agent.py (DQN)

# 数据预览

- The_battle_memory
    - Genichiro_Ashina
        - training_data-1.npy (演示用数据集)

In [ ]:
import os

import cv2
import numpy as np

boss = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', boss, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)
print(data.shape)    # 表示当前演示的数据集由200个数据组成，其中每个数据又由图像与按键记录一一对应的数据组成

## 单个数据的展示

In [ ]:
screen = data[0][0]    # 选取了第一个数据的图像部分
print('图像', screen, screen.shape, screen.dtype)

In [ ]:
action_value = data[0][1]    # 选取了第一个数据的按键记录部分
print('按键记录', action_value)

## 以视频的形式展示数据

In [ ]:
Remaining = len(data)
for screen, action_value in data:
    if   action_value == [1,0,0,0,0]:
        action = '攻击'
    elif action_value == [0,1,0,0,0]:
        action = '弹反'
    elif action_value == [0,0,1,0,0]:
        action = '垫步'
    elif action_value == [0,0,0,1,0]:
        action = '跳跃'
    elif action_value == [0,0,0,0,1]:
        action = '其他'

    cv2.imshow('screen', screen)
    cv2.waitKey(10)
    
    Remaining -= 1
    print(f'\r 剩余: {Remaining:>4}, 动作:{action:<11}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

# pysekiro 模块解析

## 按键部分
- keys.py (键盘按键的信息)
- get_keys.py (捕获键盘的按键)
- direct_keys.py (控制键盘的按键)
- actions.py (动作控制)

### keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/keys.py)

**注：这部分不是由我编写的，所以我只从应用的角度说明**

这个文件不是用来直接使用，而是给我们查找相应的键对应的值的。  
其中，检测按键的是 virtual keys，输出按键的是 direct keys，相应代码在第 146 ~ 254 和 35 ~ 143 行。
使用如下

### get_keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_keys.py)

**注：这部分不是由我编写的，所以我只从应用的角度说明**

检测按键的部分，用的是 virtual keys 。

其中的字典 vk 是从 keys.py 中截取的部分键值对，然后，调用 key_check() 时就会检测当前正在按的键

里面的按键我已经定义好了，如果你想定义更多的按键，可以在 **keys.py** 中第 146 ~ 254行找到相应的按键然后添加进去

运行以下代码体验一下，按 P 停止

In [ ]:
from pysekiro.get_keys import key_check
while True:
    keys = key_check()
    print(f'\r{str(keys):<60}', end = '')
    if 'P' in keys:
        break

### direct_keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/direct_keys.py)

**注：这部分不是由我编写的，所以我只从应用的角度说明**

PressKey 压键  
ReleaseKey 松键  

具体用法看 actions.py

### actions.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/actions.py)

输出按键的部分，用的是 direct keys 

#### 定义执行相应动作和函数

和 get_keys.py 不同的是，这里不用字典处理，而是采用直接赋值的方式。  
因为从 direct_keys.py 导入的 PressKey() 和 ReleaseKey() 参数都是 hexKeyCode ，直接输入相应的 十六进制 映射到按键就好了。  
赋值给相应的变量只是为了好认而已。

In [ ]:
# e.g.
from pysekiro.direct_keys import PressKey, ReleaseKey
import time

W = 0x11
def Move_Forward():
    print('移动 前')
    PressKey(W)      # 按键
    time.sleep(1)    # 按键时长（控制长按短按）
    ReleaseKey(W)    # 松键

for _ in range(5):
    Move_Forward()

调用其中的某个函数时，游戏就会接收到程序发送过来的按键信号，然后驱动你控制的人物做出相应的动作

#### act(values)

接受到动作所对应的值（比如说这里 0 代表 攻击），然后把其对应的动作的函数名传递给局部变量 act ，再用多线程执行动作。  
如果你想要让它能有更多的动作选择，可以定义好相应的动作的函数（定义如上），然后再用 elif 补充，不过，这可以需要你数据的支持和模型的支持。  
就是说如果你在数据收集时只收集了5类动作，模型训练的时候也是，动作空间数量为5，那么你定义的6个及6个以上的动作时将会有一部分动作是无效的。  
~~~python
# 根据 collect_data.py
def act(values):
    
    if   values == 0:
        act = Attack     # 攻击
    elif values == 1:
        act = Deflect    # 弹反
    elif values == 2:
        act = Step_Dodge # 垫步
    elif values == 3:
        act = Jump       # 跳跃
    elif values == 4:
        act = Move_Forward # 其他

    act_process = threading.Thread(target=act)
    act_process.start()
~~~

## 图像部分
- grab_screen.py (屏幕图像抓取)
- get_vertices.py (顶点位置获取)
- get_status.py (状态获取)

### grab_screen.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/grab_screen.py)

**注：这部分中主题部分不是由我编写的，所以我只从我编写的部分说明**  

#### 主体部分 grab_screen(region=None)

这部分某个大佬写的，这个函数获取指定区域的图像然后返回原图，其中，我的修改是，把返回值改成了灰度图

#### 我处理的部分 get_screen()

我指定了游戏窗口的位置，指定了图像缩放的大小

In [ ]:
import cv2
from pysekiro.grab_screen import get_screen

screen = get_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

print('图像', screen, screen.shape, screen.dtype)

实际上，数据集中单个数据里的图像部分就是由这个函数获取的。

### get_vertices.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_vertices.py)

其实就是我无法写出一个自动把血条、架势条抠出来的程序，所以才写的这个半自动的程序。  
这个程序的主要作用就是，在图像上点击然后自动返回相关的位置信息。  
可以看一下演示的gif  
![demo.gif](https://github.com/ricagj/pysekiro_with_RL/blob/main/imgs/demo.gif)

如图所示，在弹出来的窗口中用鼠标左键按顺序依次点击左下，左上，右上，右下，然后按键盘上的“ESC”键，就会自动返回 x, x_w, y, y_h, vertices  
**图像的原点在左上角**
- x 代表 x轴的起始位置，也是图像的最左边
- y 代表 y轴的起始位置，也是图像的最上边
- x_w 代表 x轴的终止位置，也是图像的最右边
- y_h 代表 y轴的终止位置，也是图像的最下边
- vertices = [[x, y_h], [x, y], [x_w, y], [x_w, y_h]] (依次代表图像的左下角、左上角、右上角和右下角的位置。)

#### 函数解释

##### on_EVENT_LBUTTONDOWN(event, x, y, flags, param)

vertices 正是由下面这些代码获取的
~~~python
cv2.setMouseCallback("window", on_EVENT_LBUTTONDOWN)
while True:
    cv2.imshow("window", img)
    if cv2.waitKey(0)&0xFF==27:
        break
~~~
具体用法看下面

##### GrabCut_ROI(img, vertices)

其实就是展示抠图的效果的，看上面的 demo 就知道了，区域内保留，区域外全黑（像素点的值等于0）。

##### standardize(vertices)

鼠标点击出来的位置只是大概的，再怎么精细也只是类似矩形，而不是矩形，所以靠这个函数处理成最标准的矩形。  
处理标准：尽量取最大范围

##### roi(img, x, x_w, y, y_h)

只保留区域内，区域外的全部不要

##### get_vertices(img)

直接上代码解释

~~~python
def get_vertices(img):
    
    global vertices
    vertices = []

    print('Press "ESC" to quit. ') # 按ESC键离开。
    cv2.namedWindow("window", cv2.WINDOW_NORMAL)    # 使窗口可以调整，比如图片太小，就调整窗口
    cv2.setMouseCallback("window", on_EVENT_LBUTTONDOWN)
    while True:
        cv2.imshow("window", img)
        if cv2.waitKey(0)&0xFF==27:
            break
    cv2.destroyAllWindows()

    if len(vertices) != 4:    # 矩形只有4个顶点
        print("vertices number not match")
        return -1

    x, x_w, y, y_h, vertices = standardize(vertices)    # 矩形化

    cv2.imshow('img', img)
    cv2.imshow('GrabCut_ROI(img)', GrabCut_ROI(img, [np.array(vertices)]))    # GrabCut_ROI 效果
    cv2.imshow('roi(img)', roi(img, x, x_w, y, y_h))    # roi 效果

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f'\n x={x}, x_w={x_w}, y={y}, y_h={y_h}, vertices={vertices} \n')
~~~

#### 演示

In [ ]:
import cv2
from pysekiro.get_vertices import get_vertices

img = cv2.imread(".\imgs\demo.png", 0)
get_vertices(img)

其实最后需要用的只有 x, x_w, y, y_h

### get_status.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_status.py)

#### 演示

In [ ]:
import os
import cv2
import numpy as np
from pysekiro.get_status import get_status

target = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', target, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)

Remaining = len(data)

for screen in data[:,0]:

    cv2.imshow('screen', screen)

    Self_HP, Self_Posture, Target_HP, Target_Posture = get_status(screen)

    Remaining -= 1
    print(f'\r Remaining: {Remaining:<6}, Self HP: {Self_HP:>3}, Self Posture: {Self_Posture:>3}, Target HP: {Target_HP:>3}, Target Posture: {Target_Posture:>3}', end='')
    cv2.waitKey(10)
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

#### get_status()

~~~python
Self_HP, Self_Posture, Target_HP, Target_Posture = get_status(screen)
~~~

~~~python
def get_status(img):
    return [get_Sekiro_HP(img), get_Sekiro_Posture(img), get_Target_HP(img), get_Target_Posture(img)]
~~~

##### 其中四个函数的原型

~~~python
def get_Self_HP(img):
    img_roi = roi(img, x=29, x_w=182, y=246, y_h=246+1)[0]
    retval, img_th = cv2.threshold(img_roi, 60, 255, cv2.THRESH_TOZERO)    # 低于60的像素点的值设置为0
    retval, img_th = cv2.threshold(img_th, 80, 255, cv2.cv2.THRESH_TOZERO_INV)    # 高于80的像素点的值设置为0
    img_th = np.reshape(img_th, (img_roi.shape))
    Self_HP = get_value(img_th)    # 获取数值
    return Self_HP
~~~

~~~python
def get_Target_HP(img):
    img_roi = roi(img, x=29, x_w=130, y=25, y_h=25+1)[0]
    retval, img_th = cv2.threshold(img_roi, 40, 255, cv2.THRESH_TOZERO)    # 低于40的像素点的值设置为0
    retval, img_th = cv2.threshold(img_th, 80, 255, cv2.cv2.THRESH_TOZERO_INV)    # 高于80的像素点的值设置为0
    img_th = np.reshape(img_th, (img_roi.shape))
    Target_HP = get_value(img_th)    # 获取数值
    return Target_HP
~~~

架势条不像生命条一样一直显示着，架势为0时消失，架势不为0时才出现，如果不设置开启条件，那很容易得到错误的架势信息

~~~python
def get_Self_Posture(img):
    img_roi = roi(img, x=240, x_w=290, y=234, y_h=234+1)[0]
    retval, img_th = cv2.threshold(img_roi, 100, 255, cv2.THRESH_TOZERO)    # 低于100的像素点的值设置为0
    img_th = np.reshape(img_th, (img_roi.shape))
    
    if int(img_th[0]) - int(img_th[1]) > 15:    # 开启条件
        if img_th[1] in range(100, 125) and img_th[0] in range(145, 165):
            Self_Posture = get_value(img_th)
        elif img_th[1] in range(135, 160) and img_th[0] in range(180, 220):
            Self_Posture = get_value(img_th)
        elif img_th[1] in range(160, 230) and img_th[0] in range(200, 250):
            Self_Posture = get_value(img_th)
        else:
            Self_Posture = 0
    else:
        Self_Posture = 0
    return Self_Posture
~~~

~~~python
def get_Target_Posture(img):
    img_roi = roi(img, x=240, x_w=327, y=17, y_h=17+1)[0]
    retval, img_th = cv2.threshold(img_roi, 100, 255, cv2.THRESH_TOZERO)    # 低于100的像素点的值设置为0
    img_th = np.reshape(img_th, (img_roi.shape))
    
    if int(img_th[0]) - int(img_th[1]) > 15:    # 开启条件
        if img_th[1] in range(100, 125) and img_th[0] in range(175, 222):
            Target_Posture = get_value(img_th)
        elif img_th[1] in range(125, 210) and img_th[0] in range(190, 250):
            Target_Posture = get_value(img_th)
        else:
            Target_Posture = 0
    else:
        Target_Posture = 0
    return Target_Posture
~~~

##### 获取数值的部分

范围外的已经处理成0了，所以计算第一个出现的0的左边有多少个非0值，得到的结果就是我们需要的那个状态值

~~~python
def get_value(target_img):
    count = 0
    for i in range(len(target_img)-1):
        cur_pixel = int(target_img[i])
        if cur_pixel == 0:
            break
        count += 1
    return count
~~~

##### 演示

接下来看下面的代码运行的结果

In [ ]:
import os
import cv2
import numpy as np
from pysekiro.get_vertices import roi

def get_value(target_img):
    count = 0
    for i in range(len(target_img)-1):
        cur_pixel = int(target_img[i])
        if cur_pixel == 0:
            break
        count += 1
    return count

def get_Target_HP(img):
    img_roi = roi(img, x=29, x_w=130, y=25, y_h=25+1)[0]
    retval, img_th = cv2.threshold(img_roi, 40, 255, cv2.THRESH_TOZERO)
    retval, img_th = cv2.threshold(img_th, 80, 255, cv2.cv2.THRESH_TOZERO_INV)
    img_th = np.reshape(img_th, (img_roi.shape))
    Target_HP = get_value(img_th)
    print('\n', img_th)
    return Target_HP

boss = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', boss, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)

for screen in data[[89, 91, 199],0]:

    Target_HP = get_Target_HP(screen)

    print(f'Target_HP: {Target_HP:>4}')

## collect_data.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/collect_data.py)

搜集数据顺带评分

### get_output()

对按键信息进行独热编码  
注意，这部分的定义非常重要，后面的很多设置都与它相适应

### class Data_collection

#### save_data

保存数据，统一名称为 f'training_data-{n}.npy'，例如 training_data-123.npy  
从1开始检测文件名是否存在于保存路径，直到检测到不存在就保存

#### collect_data

核心代码
~~~python
screen = get_screen()    # 获取屏幕图像
action = get_output()    # 获取按键输出
self.dataset.append([screen, action])    # 图像和输出打包在一起，保证一一对应

status = get_status(screen)
reward = self.reward_system.get_reward(status, np.argmax(action))    # 根据当前状态和动作，计算 reward
~~~

## adjustment.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/adjustment.py)

由collect_data.py改编而来，只做检测不做收集和评分

## model.py

我定义的是一个简化版的残差神经网络，大家可以根据自己的情况定义模型

## train_with_dl.py

核心代码
~~~python
for i in range(start, end+1):    # 读取一个数据集并学习，然后读取下一个数据集并学习，以此类推
    filename = f'training_data-{i}.npy'
    data_path = os.path.join('The_battle_memory', target, filename)
    data = np.load(data_path, allow_pickle=True)

    # 数据集处理成预训练格式
    X = np.array([roi(i[0], x, x_w, y, y_h) for i in data]).reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
    Y = np.array([i[1] for i in data])

    # 训练模型，然后保存
    model.fit(X, Y, batch_size=batch_size, epochs=epochs, verbose=1)
    model.save_weights(model_weights)
~~~

## Agent.py

### RewardSystem

#### 计算 reward 的原理

reward = sum((现在的状态 - 过去的状态) * 正负强化的权重)

**注：当前设置的计算方法比较简单，所以可能会影响训练的效果，以后有空我再考虑更新更合理的计算公式**

#### 正负强化的权重

设置 正强化 和 负强化  
由于计算 reward 的算法是 现在的状态减去过去的状态，所以  
自身生命与reward呈正相关；自身架势与reward呈负相关；目标生命与reward呈负相关；目标架势与reward呈正相关

|   类型   | 状态 | reward | 权重正负 |
| :----: | :----: | :----: | :----: |
| 自身生命 |  +   |   +    |    +    |
| 自身生命 |  -   |   -    |    +    |
| 自身架势 |  +   |   -    |    -    |
| 自身架势 |  -   |   +    |    -    |
| 目标生命 |  +   |   -    |    -    |
| 目标生命 |  -   |   +    |    -    |
| 目标架势 |  +   |   +    |    +    |
| 目标架势 |  -   |   -    |    +    |

e.g. 自身生命 152 -> 131  
131 - 152 = -21 代表自身生命值减少了21 ，所以得到的 reward 应该是负数，这时候权重为正数就正合适  
e.g. 自身生命 21  ->  97  
97  -  21 =  76 代表自身生命值增加了76 ，所以得到的 reward 应该是正数，这时候权重为正数就正合适  
e.g. 目标生命 100 -> 98  
98 - 100 = -2   代表目标生命值减少了2  ，所以得到的 reward 应该是正数，这时候权重为负数就正合适  

~~~python
self.reward_weights = [0.1, -0.1, -0.1, 0.1]
~~~

### DQNReplayer

https://github.com/ZhiqingXiao/rl-book/blob/master/chapter10_atari/BreakoutDeterministic-v4_tf.ipynb  
从这里copy过来的，超级好用

### Sekiro_Agent

~~~python
self.n_action = n_action    # 动作数量

self.gamma = gamma    # 奖励衰减 默认 0.99

self.batch_size = batch_size                    # 样本抽取数量 默认 8
self.replay_memory_size = replay_memory_size    # 记忆容量 默认 50000

self.epsilon = epsilon                                # 探索参数 默认 1.0
self.epsilon_decrease_rate = epsilon_decrease_rate    # 探索衰减率 默认 0.999

self.model_weights = model_weights    # 指定读取的模型参数的路径 默认 None

self.evaluate_net = self.build_network()    # 评估网络
self.target_net = self.build_network()      # 目标网络
self.reward_system = RewardSystem()                # 奖惩系统
self.replayer = DQNReplayer(self.replay_memory_size)    # 经验回放
~~~

#### 评估网络和目标网络的构建方法
~~~python
def build_network(self):
    model = resnet(ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT,
        outputs = self.n_action
    )
    if self.model_weights:
        if os.path.exists(self.model_weights):
            model.load_weights(self.model_weights)
            print('Load ' + self.model_weights)
        else:
            print('Nothing to load')

    return model
~~~

#### 行为选择方法
~~~python
def choose_action(self, screen, train=False):
    r = 1
    if train:
        r = np.random.rand()

    if r < self.epsilon:
        q_values = np.random.randint(self.n_action)
        self.epsilon *= self.epsilon_decrease_rate
    else:
        screen = roi(screen, x, x_w, y, y_h)
        q_values = self.evaluate_net.predict([screen.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)])[0]
        q_values = np.argmax(q_values)

    act(q_values)
    return q_values
~~~

#### 学习
~~~python
def learn(self):

    # 经验回放
    screens, actions, rewards, next_screens = self.replayer.sample(self.batch_size)

    screens = screens.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
    next_screens = next_screens.reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)

    next_qs = self.target_net.predict(next_screens)
    next_max_qs = next_qs.max(axis=-1)
    targets = self.evaluate_net.predict(screens)
    targets[range(self.batch_size), actions] = rewards + self.gamma * next_max_qs

    self.evaluate_net.fit(screens, targets, verbose=0)
~~~

#### 更新目标网络权重方法
~~~python
def update_target_network(self, load_path=TMP_WEIGHTS):
    self.target_net.load_weights(load_path)
~~~

#### 保存评估网络权重方法
~~~python
def save_evaluate_network(self, save_path=TMP_WEIGHTS):
    try:
        self.evaluate_net.save_weights(save_path)
    except:
        print('save weights faild!!!')
~~~